## Bash and Magic Commands

Colab runs in a Linux environment, and you can access terminal commands with !

# Recitation 0G: Available Compute and Google Colab

Google Colab is a Jupyter Notebook service that can be accessed directly in the browser.
*   Also provides access to computing resources, including the Tesla T4 GPU
*   To access more powerful GPUs, you can choose to pay for Google Colab Pro or Pro+ (https://colab.research.google.com/signup)

To access Google Colab, just navigate to the link: https://colab.research.google.com/. You can also create a Google Colab notebook directly in your Google Drive.

This recitation assumes basic knowledge of using Jupyter Notebooks, so please familiarize yourself with it if you haven't already.


In [1]:
!nvidia-smi # allows you to view and monitor your current GPU

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
!pip install torch

In [ ]:
!ls
# !cd
# !mkdir

Below are some magic commands.

In [ ]:
# %time
# %env
# %matplotlib inline
# %debug

## Runtime

In [2]:
import torch
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: ", DEVICE)

Device:  cpu


### Utilizing Free GPU/TPU Resources
#### Changing Runtime
- Runtime > Change runtime type
- Select GPU

#### GPUs: Training Time of ResNet50
- T4: 1x Speedup (Baseline)
- A100: 10x Speedup (Comparing to T4)
- TPU: TPU is a completely different architecture and require many training constraints

## Managing Your Files

### Mounting to Google Drive
You can mount to Google Drive, which will allow you to save files, even when your runtime ends.

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Saving/Loading files - Model checkpoints

In [3]:
# Example MLP
import torch
import torch.nn as nn

class MLP(nn.Module):

    def __init__(self, size):
        super(MLP, self).__init__()

        self.layers = []
        for in_dim, out_dim in zip(size[:-2], size[1:-1]):
          self.layers.extend([
              nn.Linear(in_dim, out_dim),
              nn.ReLU(),
              nn.BatchNorm1d(out_dim),
              nn.Dropout(0.5),
          ])
        self.layers.append(nn.Linear(size[-2], size[-1]))
        self.model = nn.Sequential(*self.layers)
        self.model.apply(self.init_param)

    def init_param(self, param):
      if type(param) == nn.Linear:
        nn.init.xavier_uniform_(param.weight)

    def forward(self, x):
      return self.model(x)

model = MLP([40, 2048, 512, 256, 71])

In [ ]:
!mkdir /content/drive/MyDrive/checkpoints

MODEL_SAVE_PATH = "/content/drive/MyDrive/checkpoints/checkpoint.pt"

torch.save({
  'epoch': 10,
  'model_state_dict': model.state_dict(),
  'loss': 0.001,
}, MODEL_SAVE_PATH)

### Managing Dataset

Obtaining dataset
- Kaggle Command
- Manually uploading

DO NOT directly store dataset in Google Drive, instead...
- Download/uploading dataset every time
- Move dataset from Google Drive into content folder
- Connect to GCP or AWS


In [4]:
#downloads dataset from kaggle

!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"<KAGGLE USERNAME>","key":"<API KEY>"}')
    # Put your kaggle username & key here
!chmod 600 /root/.kaggle/kaggle.json

! kaggle competitions download -c 11785-sp23-intro-to-colab
! unzip /content/competitions/11785-sp23-intro-to-colab/11785-sp23-intro-to-colab.zip -d /content

     ---------------------------------------- 59.2/59.2 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for kaggle: filename=kaggle-1.5.8-py3-none-any.whl size=73261 sha256=6244ffa9def59cfecde8075085c27f05a5301cae18a63c3bd25945b2dee000f4
  Stored in directory: c:\users\pontu\appdata\local\pip\cache\wheels\de\f7\d8\c3902cacb7e62cb611b1ad343d7cc07f42f7eb76ae3a52f3d1
Successfully built kaggle


The syntax of the command is incorrect.


FileNotFoundError: [Errno 2] No such file or directory: '/root/.kaggle/kaggle.json'

### Restart session vs restart runtime


Restart session
- Runtime > Restart session
- Clears all session variables

Restart runtime
- Runtime > Disconnect and delete runtime
- Deletes session
- Lose files in content folder
- Switching GPUs will also delete current runtime

# Colab Pro

- Longer session runtime, reducing risk of timeout
- Priority access to GPU
- Increased storage
- Supposedly background execution
